In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score


from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import os

from sklearn.metrics import classification_report,confusion_matrix,f1_score
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score

In [ ]:
data = pd.read_csv('C:\\Users\\14437\\Downloads\\full_data.csv')
data.head()

In [ ]:
data_clean = data[data['age']>25]

In [ ]:
data_clean = data_clean.replace({'gender': {'Female': 0, 'Male': 1}})
data_clean['stroke'].value_counts().to_frame().T

In [ ]:

data_clean = data_clean.replace({'ever_married': {'Yes': 1, 'No': 0}})

In [ ]:
data2 = pd.get_dummies(data_clean, columns=["work_type", "Residence_type","smoking_status"], prefix=["work", "residence","smoke"])

In [ ]:
target = 'stroke'
X = data2.loc[:,data2.columns!=target]
y = data2.loc[:,data2.columns==target]

In [ ]:
import seaborn as sns
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

X.shape, y.shape
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, shuffle=True)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_gini = DecisionTreeClassifier()
clf_gini.fit(X_train, y_train)

In [ ]:
y_pred_gini = clf_gini.predict(X_test)
accuracy_score(y_test, y_pred_gini)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(y_test, y_pred_gini)))

In [ ]:
y_pred_train_gini = clf_gini.predict(X_train)

y_pred_train_gini

In [ ]:
print('Training-set accuracy score:{0:0.4f}'. format(accuracy_score(y_train, y_pred_train_gini)))

In [ ]:
print('Training set score: {:.4f}'.format(clf_gini.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf_gini.score(X_test, y_test)))

In [ ]:

from matplotlib.pyplot import *
plt.figure(figsize=(12,8))

from sklearn import tree

tree.plot_tree(clf_gini.fit(X_train, y_train)) 


In [ ]:
# CHANGING MAX_DEPTH
score_train=[]
score_test=[]

for i in range(1,100):
    dtree_md = DecisionTreeClassifier(criterion= "gini", max_depth=i)
    dtree_md.fit(X_train,y_train)
    
    score_train.append(dtree_md.score(X_train,y_train)*100)
    score_test.append(dtree_md.score(X_test,y_test)*100)
    
score_train_array=np.asarray(score_train)
score_test_array=np.asarray(score_test)
plt.figure(figsize=(20,6))
plt.title("Validation Curve for Decision Tree")
plt.plot(range(1,100),score_train_array,color='orange', label="Training Accuracy")
plt.plot(range(1,100),score_test_array,color='green',label="Testing Accuracy")

plt.legend()
plt.xlabel('max_depth')
plt.ylabel('Accuracy Percentage')
plt.show()

In [ ]:
path = clf_gini.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities
print(ccp_alphas)

In [ ]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)

In [ ]:
clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]
node_counts = [clf.tree_.node_count for clf in clfs]
depth = [clf.tree_.max_depth for clf in clfs]

In [ ]:
train_acc = []
test_acc = []
for c in clfs:
    y_train_pred = c.predict(X_train)
    y_test_pred = c.predict(X_test)
    train_acc.append(accuracy_score(y_train,y_train_pred))
    test_acc.append(accuracy_score(y_test,y_test_pred))


In [ ]:
from sklearn.metrics import plot_confusion_matrix
clf_ = tree.DecisionTreeClassifier(random_state=0,ccp_alpha=0.020)
clf_.fit(X_train,y_train)
y_train_pred = clf_.predict(X_train)
y_test_pred = clf_.predict(X_test)
accuracy_score(y_test, y_test_pred)

In [ ]:

from matplotlib.pyplot import *
plt.figure(figsize=(12,8))

from sklearn import tree

tree.plot_tree(clf_.fit(X_train, y_train)) 


In [ ]:
print('Training set score: {:.4f}'.format(clf_.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf_.score(X_test, y_test)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as KMeans
from sklearn import datasets
from sklearn.model_selection import learning_curve
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=10)
sizes, training_scores, testing_scores = learning_curve(clf_gini, X, y, cv=10, scoring='accuracy', train_sizes=np.linspace(0.01, 1.0, 50))

In [ ]:
train_mean = np.mean(training_scores, axis=1)
train_std = np.std(training_scores, axis=1)

test_mean = np.mean(testing_scores, axis=1)
test_std = np.std(testing_scores, axis=1)

In [ ]:
plt.subplots(1, figsize=(10,10))
plt.plot(sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(sizes, test_mean, color="#111111", label="Cross-validation score")
plt.fill_between(sizes, train_mean - train_std, train_mean + train_std, color="red")
plt.fill_between(sizes, test_mean - test_std, test_mean + test_std, color="green")

plt.title("Learning Curve for Decision Tree Classifier")
plt.xlabel("Testing Set Size"), plt.ylabel("Accuracy  Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
# CHANGING MAX_DEPTH
score_train=[]
score_test=[]

for i in range(2,200):
    dtree_md = DecisionTreeClassifier(criterion= "gini", max_leaf_nodes=i)
    dtree_md.fit(X_train,y_train)
    
    score_train.append(dtree_md.score(X_train,y_train)*100)
    score_test.append(dtree_md.score(X_test,y_test)*100)
    
score_train_array=np.asarray(score_train)
score_test_array=np.asarray(score_test)
plt.figure(figsize=(20,6))
plt.title("Validation Curve for Decision Tree")
plt.plot(range(2,200),score_train_array,color='orange', label="Training Accuracy")
plt.plot(range(2,200),score_test_array,color='green',label="Testing Accuracy")

plt.legend()
plt.xlabel('Max Leaf Nodes')
plt.ylabel('Accuracy Percentage')
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
#making the instance
model= DecisionTreeClassifier(random_state=1234)
#Hyper Parameters Set
params = {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
          'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11],
          "max_depth":[1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40],
          'random_state':[123]}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=-1)
#Learning
model1.fit(X_train,y_train)
#The best hyper parameters set
print("Best Hyper Parameters:",model1.best_params_)
#Prediction
prediction=model1.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))

In [ ]:
best_clf = model1.best_estimator_


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as KMeans
from sklearn import datasets
from sklearn.model_selection import learning_curve
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=10)
sizes, training_scores, testing_scores = learning_curve(best_clf, X, y, cv=10, scoring='accuracy', train_sizes=np.linspace(0.01, 1.0, 50))

In [ ]:
train_mean = np.mean(training_scores, axis=1)
train_std = np.std(training_scores, axis=1)

test_mean = np.mean(testing_scores, axis=1)
test_std = np.std(testing_scores, axis=1)

In [ ]:
plt.subplots(1, figsize=(10,10))
plt.plot(sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(sizes, test_mean, color="#111111", label="Cross-validation score")
plt.fill_between(sizes, train_mean - train_std, train_mean + train_std, color="red")
plt.fill_between(sizes, test_mean - test_std, test_mean + test_std, color="green")

plt.title("Learning Curve for Decision Tree Classifier")
plt.xlabel("Testing Set Size"), plt.ylabel("Accuracy  Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_digits
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import validation_curve

 
# Setting the range for the parameter (from 1 to 10)
parameters = {
    'max_depth': [1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100], 
}
 
# Calculate accuracy on training and test set using the
# gamma parameter with 5-fold cross validation
train_score, test_score = validation_curve(best_clf, X, y,
                                       param_name = "max_depth",
                                       param_range = parameters['max_depth'],
                                        cv = 5, scoring = "accuracy")
 
# Calculating mean and standard deviation of training score
mean_train_score = np.mean(train_score, axis = 1)
std_train_score = np.std(train_score, axis = 1)
 
# Calculating mean and standard deviation of testing score
mean_test_score = np.mean(test_score, axis = 1)
std_test_score = np.std(test_score, axis = 1)

In [ ]:
 
# Plot mean accuracy scores for training and testing scores
plt.plot(parameters['max_depth'], mean_train_score,
     label = "Training Score", color = 'b')
plt.plot(parameters['max_depth'], mean_test_score,
   label = "Cross Validation Score", color = 'g')
 
# Creating the plot
plt.title("Validation Curve with Decision Tree Classifier")
plt.xlabel("Kernel Type")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.legend(loc = 'best')
plt.show()

In [ ]:
best_clf = DecisionTreeClassifier(criterion='gini', max_depth=20, random_state=1)

best_clf.fit(X_train, y_train)
y_test_pred = best_clf.predict(X_test)
test_score = accuracy_score(y_test, y_test_pred)
print('%.4f' % test_score)